In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [19]:
event = pd.read_csv('medalion_data_store/bronze/events.csv')
unique_event_features = pd.read_csv('medalion_data_store/silver/unique_event_features.csv')

In [7]:
unique_event_features

,Unnamed: 0,person,ofr_id_short,tag,reward_completed,cnt_completed,cnt_received,cnt_viewed,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence
0,0,0009655768c64bdeb2e877511632db8f,ofr_C,0,0.0,0.0,1.0,1.0,2.7,0.0,0.0,0,0
1,1,0009655768c64bdeb2e877511632db8f,ofr_G,0,2.0,1.0,1.0,1.0,2.7,-1.0,4.0,0,0
2,2,0009655768c64bdeb2e877511632db8f,ofr_H,0,0.0,0.0,1.0,1.0,4.0,0.0,0.0,0,0
3,3,0009655768c64bdeb2e877511632db8f,ofr_I,0,5.0,1.0,1.0,1.0,2.0,-1.0,14.3,0,0
4,4,0009655768c64bdeb2e877511632db8f,ofr_J,0,2.0,1.0,1.0,0.0,0.0,-1.0,100.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,5.0,1.0,1.0,1.0,14.3,4.0,3.2,1,0
76273,76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,2.0,1.0,1.0,1.0,14.3,1.8,1.6,1,0
76274,76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,2.0,1.0,1.0,1.0,5.3,3.2,2.0,1,0
76275,76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,2.0,1.0,1.0,1.0,14.3,100.0,14.3,1,1


In [8]:
rank2 = unique_event_features.groupby(['ofr_id_short'])[[
    'cnt_received',
    'cnt_viewed',
    'cnt_completed',
    'influence',
    'ext_influence'
]].sum().sort_values(ascending=False, by=['influence']).reset_index()

rank2['completition_rate'] = rank2['cnt_completed'] / rank2['cnt_received']
rank2['viewed_rate'] = rank2['cnt_viewed'] / rank2['cnt_received']

rank2

,ofr_id_short,cnt_received,cnt_viewed,cnt_completed,influence,ext_influence,completition_rate,viewed_rate
0,ofr_G,7597.0,7327.0,5317.0,4664,415,0.699882,0.964460
1,ofr_F,7646.0,7337.0,5156.0,4392,457,0.674340,0.959587
2,ofr_I,7571.0,7264.0,4296.0,3559,494,0.567428,0.959451
3,ofr_B,7593.0,7298.0,3331.0,2788,375,0.438694,0.961148
4,ofr_A,7658.0,6716.0,3688.0,2629,332,0.481588,0.876991
5,ofr_D,7677.0,4171.0,4354.0,2125,269,0.567149,0.543311
6,ofr_J,7632.0,4118.0,4017.0,2111,206,0.526336,0.539570
7,ofr_E,7668.0,2663.0,3420.0,1311,79,0.446009,0.347287
8,ofr_C,7617.0,4144.0,0.0,0,0,0.000000,0.544046
9,ofr_H,7618.0,6687.0,0.0,0,0,0.000000,0.877789


In [9]:
offer_rank = rank2.sort_values(by=['ext_influence', 'completition_rate'], ascending=False)

def offer_rank_recommendation(person_id, event, offer_rank):
    products_senn = event[event['person'] == 'ffff82501cea40309d5fdd7edcca4a07']['ofr_id_short'].unique()

    products_not_senn = (rank2[~rank2['ofr_id_short']
                               .isin(products_senn)]
                               .sort_values(by=['ext_influence', 'completition_rate'], ascending=False)['ofr_id_short']
                               .tolist())
    
    return products_not_senn #.head(5)

print(offer_rank_recommendation('ffff82501cea40309d5fdd7edcca4a07', event, offer_rank))

['ofr_I', 'ofr_F', 'ofr_B', 'ofr_A', 'ofr_C', 'ofr_H']


In [3]:
user_item_profile = pd.read_csv('medalion_data_store/gold/user_item_profile.csv')
user_item_profile

,gender,age,id,became_member_on,income,bec_memb_year_month,age_group,cnt_completed_ofr_G,cnt_completed_ofr_I,cnt_completed_ofr_J,...,cnt_viewed_ofr_F,cnt_completed_ofr_B,cnt_received_ofr_A,cnt_received_ofr_B,cnt_viewed_ofr_B,cnt_completed_ofr_A,cnt_viewed_ofr_A,cnt_viewed_ofr_J,cnt_transaction,sum_transaction
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2017-02,Senior,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,20.40
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017-07,Middle,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,77.01
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,2018-07,Senior,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,14.30
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2017-05,Senior,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,7.0,159.27
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,2017-08,Senior,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,F,45,6d5f3a774f3d4714ab0c092238f3a1d7,2018-06-04,54000.0,2018-06,Middle,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,7.0,20.03
16996,M,61,2cb4f97358b841b9a9773a7aa05a9d77,2018-07-13,72000.0,2018-07,Senior,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,25.97
16997,M,49,01d26f638c274aa0b965d24cefe3183f,2017-01-26,73000.0,2017-01,Middle,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,39.74
16998,F,83,9dc1421481194dcd9400aec7c9ae6366,2016-03-07,50000.0,2016-03,Senior,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,14.0,189.67


In [4]:
user_item_matrix = (pd.concat([
    user_item_profile.drop(columns=['age_group']),
    pd.get_dummies(user_item_profile['age_group'], dummy_na=True, prefix='age').astype(int),
    pd.get_dummies(user_item_profile['gender'], dummy_na=True, prefix='gen').astype(int)
    ], axis=1)
    .drop(columns=['age', 'gender', 'became_member_on', 'bec_memb_year_month'])
    .fillna(0)
    )
user_item_matrix

,id,income,cnt_completed_ofr_G,cnt_completed_ofr_I,cnt_completed_ofr_J,cnt_received_ofr_C,cnt_received_ofr_G,cnt_received_ofr_H,cnt_received_ofr_I,cnt_received_ofr_J,...,sum_transaction,age_Adult,age_Middle,age_Senior,age_Young,age_nan,gen_F,gen_M,gen_O,gen_nan
0,68be06ca386d4c31939f3a4f0e3dd783,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,20.40,0,0,1,0,0,0,0,0,1
1,0610b486422d4921ae7d2bf64640c50b,112000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,77.01,0,1,0,0,0,1,0,0,0
2,38fe809add3b4fcf9315a9694bb96ff5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,14.30,0,0,1,0,0,0,0,0,1
3,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,159.27,0,0,1,0,0,1,0,0,0
4,a03223e636434f42ac4c3df47e8bac43,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,4.65,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,20.03,0,1,0,0,0,1,0,0,0
16996,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,25.97,0,0,1,0,0,0,1,0,0
16997,01d26f638c274aa0b965d24cefe3183f,73000.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,39.74,0,1,0,0,0,0,1,0,0
16998,9dc1421481194dcd9400aec7c9ae6366,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,189.67,0,0,1,0,0,1,0,0,0


In [5]:
# Supondo que df seja o seu DataFrame
df = user_item_matrix.drop(columns=['id'])

# Identificar os tipos de colunas
continuous_cols = ["income", "sum_transaction"]
count_cols = [col for col in df.columns if col.startswith("cnt_")]

# Inicializar escaladores
scaler_continuous = StandardScaler()
scaler_counts = MinMaxScaler()

# Aplicar escalonamento
df[continuous_cols] = scaler_continuous.fit_transform(df[continuous_cols])
df[count_cols] = scaler_counts.fit_transform(df[count_cols])

# Exibir amostra do DataFrame escalado
df.head()

,income,cnt_completed_ofr_G,cnt_completed_ofr_I,cnt_completed_ofr_J,cnt_received_ofr_C,cnt_received_ofr_G,cnt_received_ofr_H,cnt_received_ofr_I,cnt_received_ofr_J,cnt_viewed_ofr_C,...,sum_transaction,age_Adult,age_Middle,age_Senior,age_Young,age_nan,gen_F,gen_M,gen_O,gen_nan
0,-1.918295,0.25,0.000000,0.0,0.00,0.4,0.0,0.0,0.25,0.00,...,-0.667239,0,0,1,0,0,0,0,0,1
1,1.848542,0.00,0.000000,0.0,0.25,0.0,0.0,0.0,0.00,0.00,...,-0.217694,0,1,0,0,0,1,0,0,0
2,-1.918295,0.00,0.000000,0.0,0.00,0.0,0.2,0.0,0.00,0.00,...,-0.715680,0,0,1,0,0,0,0,0,1
3,1.444952,0.00,0.333333,0.0,0.00,0.0,0.2,0.2,0.00,0.00,...,0.435540,0,0,1,0,0,1,0,0,0
4,-1.918295,0.00,0.000000,0.0,0.25,0.0,0.2,0.0,0.00,0.25,...,-0.792312,0,0,1,0,0,0,0,0,1


In [11]:
user_item_matrix

,id,income,cnt_completed_ofr_G,cnt_completed_ofr_I,cnt_completed_ofr_J,cnt_received_ofr_C,cnt_received_ofr_G,cnt_received_ofr_H,cnt_received_ofr_I,cnt_received_ofr_J,...,sum_transaction,age_Adult,age_Middle,age_Senior,age_Young,age_nan,gen_F,gen_M,gen_O,gen_nan
0,68be06ca386d4c31939f3a4f0e3dd783,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,20.40,0,0,1,0,0,0,0,0,1
1,0610b486422d4921ae7d2bf64640c50b,112000.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,77.01,0,1,0,0,0,1,0,0,0
2,38fe809add3b4fcf9315a9694bb96ff5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,14.30,0,0,1,0,0,0,0,0,1
3,78afa995795e4d85b5d9ceeca43f5fef,100000.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,159.27,0,0,1,0,0,1,0,0,0
4,a03223e636434f42ac4c3df47e8bac43,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,4.65,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,6d5f3a774f3d4714ab0c092238f3a1d7,54000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,20.03,0,1,0,0,0,1,0,0,0
16996,2cb4f97358b841b9a9773a7aa05a9d77,72000.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,25.97,0,0,1,0,0,0,1,0,0
16997,01d26f638c274aa0b965d24cefe3183f,73000.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,39.74,0,1,0,0,0,0,1,0,0
16998,9dc1421481194dcd9400aec7c9ae6366,50000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,189.67,0,0,1,0,0,1,0,0,0


In [13]:
user_similarity_df.index

Index(['68be06ca386d4c31939f3a4f0e3dd783', '0610b486422d4921ae7d2bf64640c50b',
       '38fe809add3b4fcf9315a9694bb96ff5', '78afa995795e4d85b5d9ceeca43f5fef',
       'a03223e636434f42ac4c3df47e8bac43', 'e2127556f4f64592b11af22de27a7932',
       '8ec6ce2a7e7949b1bf142def7d0e0586', '68617ca6246f4fbc85e91a2a49552598',
       '389bc3fa690240e798340f5a15918d5c', '8974fc5686fe429db53ddde067b88302',
       ...
       '79edb810789c447e8d212a324b44cc16', 'a9a20fa8b5504360beb4e7c8712f8306',
       '400d0536e8794cbb855b0d882d67cbda', 'cb23b66c56f64b109d673d5e56574529',
       'c02b10e8752c4d8e9b73f918558531f7', '6d5f3a774f3d4714ab0c092238f3a1d7',
       '2cb4f97358b841b9a9773a7aa05a9d77', '01d26f638c274aa0b965d24cefe3183f',
       '9dc1421481194dcd9400aec7c9ae6366', 'e4052622e5ba45a8b96b59aba68cf068'],
      dtype='object', length=17000)

In [14]:
user_similarity_df['68be06ca386d4c31939f3a4f0e3dd783'].drop(index='68be06ca386d4c31939f3a4f0e3dd783').sort_values(ascending=False)

690deebf33ba4269a28a33698417007d    0.989096
9b2ec72a4ee04bb1bbf360642f22e7d3    0.987676
8dcbd64b4c664b5da20e8203ca26486c    0.985109
22ff97b3841c460c8cb85aabfe14ae48    0.982467
71dcdf175893482cb9d00762e20939c0    0.977433
                                      ...   
3a298dbf99ef4a23b9b46a1500e56101   -0.599888
abfccb80eb3d46afb173c85e3d335194   -0.606890
f105b8f61dda45739cd5b0d64807ec0a   -0.607579
d53a0901e66d4a5c96a1f3cda56536e1   -0.613280
368b266ba48447dfb5342504d0fd7fb0   -0.613767
Name: 68be06ca386d4c31939f3a4f0e3dd783, Length: 16999, dtype: float64

In [24]:
set(event[event['person'] == 'fbfe4e14cb5147d2b0ce619378b40069']['ofr_id_short'])

{'ofr_C', 'ofr_D', 'ofr_E', 'ofr_H'}

In [38]:
# Compute user similarity
user_similarity_df = None
user_similarity = cosine_similarity(df)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_profile['id'].tolist(), columns=user_item_profile['id'].tolist())

fbfe4e14cb5147d2b0ce619378b40069
{'ofr_H', 'ofr_C', 'ofr_E', 'ofr_D'}
{'ofr_H', 'ofr_C', 'ofr_E'}


['ofr_D']

In [41]:
def recommend_user_user(user_id, user_similarity_df, event):
    if user_id not in user_similarity_df.index:
        return []  # Se o usuário não estiver na matriz de similaridade

    # Identificar o usuário mais similar
    similar_users = user_similarity_df[user_id].drop(index=user_id).sort_values(ascending=False)
    similar_user = similar_users.index[0]
    print(f"Usuário similar encontrado: {similar_user}")

    # Identificar os produtos consumidos pelo usuário similar e pelo usuário atual
    user_products = set(event[event['person'] == similar_user]['ofr_id_short'])
    similar_user_products = set(event[event['person'] == user_id]['ofr_id_short'])

    print(f"Produtos do usuário similar ({similar_user}): {user_products}")
    print(f"Produtos do usuário atual ({user_id}): {similar_user_products}")

    # Recomendar produtos que o usuário similar consumiu, mas que o usuário atual não viu
    recommendations = list(user_products - similar_user_products)
    
    print(f"Recomendações: {recommendations}")
    return recommendations


In [42]:
r = recommend_user_user('01d26f638c274aa0b965d24cefe3183f', user_similarity_df, event)
r

Usuário similar encontrado: fbfe4e14cb5147d2b0ce619378b40069
Produtos do usuário similar (fbfe4e14cb5147d2b0ce619378b40069): {'ofr_H', 'ofr_C', 'ofr_E', 'ofr_D'}
Produtos do usuário atual (01d26f638c274aa0b965d24cefe3183f): {'ofr_H', 'ofr_C', 'ofr_E'}
Recomendações: ['ofr_D']


['ofr_D']

In [ ]:
# Content-Based Recommendations

# Selecionar características do produto para recomendação (exemplo: métricas das interações)
features = ['curiosity_vr', 'eagerness_cv', 'overall_cr', 'tran_amoun_mean']
df_produto = df.groupby('ofr_id_short')[features].mean().reset_index()

# Normalizar as características
scaler = StandardScaler()
df_produto_scaled = df_produto.copy()
df_produto_scaled[features] = scaler.fit_transform(df_produto[features])

# Calcular similaridade entre produtos
product_similarity = cosine_similarity(df_produto_scaled[features])
product_similarity_df = pd.DataFrame(product_similarity, index=df_produto_scaled['ofr_id_short'], columns=df_produto_scaled['ofr_id_short'])

# Função para recomendar produtos similares com base nos produtos já consumidos pelo usuário
def recomendar_content_based(usuario_id, df, product_similarity_df):
    produtos_vistos = df[df['person'] == usuario_id]['ofr_id_short'].unique()
    recomendacoes = []

    for produto in produtos_vistos:
        if produto in product_similarity_df.index:
            similares = product_similarity_df[produto].sort_values(ascending=False).index[1:4].tolist()
            recomendacoes.extend(similares)

    # Retornar top 5 recomendações únicas
    return list(set(recomendacoes))[:5]

# Exemplo de recomendação para um usuário específico
print(recomendar_content_based('user_123', df, product_similarity_df))